In [1]:
'''
This notebook runs with pop_func function.
It is a notebook that takes location_lad.csv file, downloaded from ONS nomis website.
https://www.nomisweb.co.uk/query/construct/components/stdListComponent.asp?menuopt=12&subcomp=100
the above link takes you to the website to query ons population data to download relevant population per LAD per 5 year age bands.
How I did this for London
In Geography: select some local authority (districts)>choose region>london>tick all
In Date: select years 2018 to 2023
In age: scroll to individual ages and 5 year bands>select age 18, age 19 and age bands 20-85+
In sex: select: total
Confirm selection in summary of selections
format layout> as csv and download
The resulting output is london_lad.csv below.
The london_lad.csv is passed through the pipeline below to produce lon_pop_18_23.csv which is used in 2024 population prediction in SDE notebook
used to age standardise.
'''

'\nThis notebook runs with pop_func function.\nIt is a notebook that takes location_lad.csv file, downloaded from ONS nomis website.\nhttps://www.nomisweb.co.uk/query/construct/components/stdListComponent.asp?menuopt=12&subcomp=100\nthe above link takes you to the website to query ons population data to download relevant population per LAD per 5 year age bands.\nHow I did this for London\nIn Geography: select some local authority (districts)>choose region>london>tick all\nIn Date: select years 2018 to 2023\nIn age: scroll to individual ages and 5 year bands>select age 18, age 19 and age bands 20-85+\nIn sex: select: total\nConfirm selection in summary of selections\nformat layout> as csv and download\nThe resulting output is london_lad.csv below.\n'

In [1]:
import pandas as pd

In [2]:
%run pop_func.py

In [3]:

# Load the full CSV
df = pd.read_csv("london_lad.csv")  # replace with your actual filename

# Preview the columns
print(df.columns)

Index(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13',
       'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16'],
      dtype='object')


In [11]:
# Identify rows where a new year starts (e.g., look for rows with "Date :" or similar markers)
year_rows = df[df.iloc[:, 0].str.contains("Date", na=False)].index.tolist()

# Add the end index for the final block
year_rows.append(len(df))

# Prepare a dictionary to store data for each year
year_tables = {}

# Loop through each year block
for i in range(len(year_rows) - 1):
    start = year_rows[i] + 1
    end = year_rows[i + 1]

    # Slice the dataframe for that year
    year_df = df.iloc[start:end].copy()
    
    # Remove rows that are NaN or empty
    year_df.dropna(how='all', inplace=True)
    
    # The first row in this block is likely the column header
    year_df.columns = year_df.iloc[0]
    year_df = year_df[1:]

    # Add LAD name column
    year_df = year_df.rename(columns={year_df.columns[0]: "lad"})
    
    # Save it in dictionary
    year = df.iloc[year_rows[i], 1]  # Assuming "Date :" is in col 0 and the year in col 1
    year_tables[year] = year_df.reset_index(drop=True)



In [12]:
for year, table in year_tables.items():
    table.to_csv(f"df_{year}.csv", index=False)

In [39]:
# Example: list of DataFrames
dfs = ['df_2018', 'df_2019', 'df_2020', 'df_2021', 'df_2022', 'df_2023']
table= []
for df in dfs:
    df =pd.read_csv(f"{df}.csv")
    table.append(df)

cleaned_tables = []

for i, df in enumerate(table):
    #here we clean the table to remove parts of the table that should not be in the final table
    #we are removing rows 1 and last 2 rows if not df5, because df5 does not have an extra last row we only remove the top row
    df.columns = df.iloc[0]  # Set first row as header
    if i != 5:
        df = df[1:-2]        # Drop first and last two rows
    else:
        df = df[1:]          # Drop only the first row
    df = df.reset_index(drop=True)
    cleaned_tables.append(df)

for df in cleaned_tables:
    df.rename(columns={df.columns[0]: 'lad'}, inplace=True)    

# cleaned_tables[1]

In [38]:
transposed_tables = [df.T for df in cleaned_tables]

# transposed_tables[1]

In [44]:
final_tables = []

for df in cleaned_tables:
    df = df.copy()
    df = df.set_index('lad')      # Make 'lad' the index temporarily to preserve it
    df_t = df.T.reset_index()     # Transpose and bring index back as a column
    df_t.columns.name = None      # Remove any name from columns (optional)
    final_tables.append(df_t)

for df in final_tables:
    df.rename(columns={df.columns[0]: 'Age'}, inplace=True)
    
# final_tables[5]
years = range(2018, 2024)

for i, year in enumerate(years):
    df = final_tables[i]
    df.to_csv(f"df_{year}.csv", index=False)

In [51]:
# How to use
file_paths = [f"df_{year}.csv" for year in range(2018, 2024)]
lad_code_map = {
    "Barking and Dagenham": "E09000002",
    "Barnet": "E09000003",
    "Bexley": "E09000004",
    "Brent": "E09000005",
    "Bromley": "E09000006",
    "Camden": "E09000007",
    "City of London": "E09000001",
    "Croydon": "E09000008",
    "Ealing": "E09000009",
    "Enfield": "E09000010",
    "Greenwich": "E09000011",
    "Hackney": "E09000012",
    "Hammersmith and Fulham": "E09000013",
    "Haringey": "E09000014",
    "Harrow": "E09000015",
    "Havering": "E09000016",
    "Hillingdon": "E09000017",
    "Hounslow": "E09000018",
    "Islington": "E09000019",
    "Kensington and Chelsea": "E09000020",
    "Kingston upon Thames": "E09000021",
    "Lambeth": "E09000022",
    "Lewisham": "E09000023",
    "Merton": "E09000024",
    "Newham": "E09000025",
    "Redbridge": "E09000026",
    "Richmond upon Thames": "E09000027",
    "Southwark": "E09000028",
    "Sutton": "E09000029",
    "Tower Hamlets": "E09000030",
    "Waltham Forest": "E09000031",
    "Wandsworth": "E09000032",
    "Westminster": "E09000033"
}

df_long = combine_pop_data(file_paths)
transform_and_save_population_data(df_long, lad_code_map, "lon_pop_18_23.csv")